In [18]:
import sys
sys.path.append('../src-py/')
from tqdm import tqdm
tqdm.pandas()
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [19]:
from flair.data import Corpus, Sentence
from flair.datasets import ColumnCorpus    
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

In [20]:
from project_debater_api import *

In [21]:
data_folder = '../../../data-ceph/arguana/arg-generation/claim-target-tagger/data/ibm_ds/'
model_folder = '../../../data-ceph/arguana/arg-generation/claim-target-tagger/model'

### Train a Target tagger on IBM dataset:

In [5]:
columns = {0: 'text', 1: 'pos', 2: 'ct'}
# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train_ds.tsv',
                              test_file='test_ds.tsv')

2022-05-01 16:29:04,885 Reading data from ../../../data-ceph/arguana/arg-generation/claim-target-tagger/data/ibm_ds
2022-05-01 16:29:04,886 Train: ../../../data-ceph/arguana/arg-generation/claim-target-tagger/data/ibm_ds/train_ds.tsv
2022-05-01 16:29:04,887 Dev: None
2022-05-01 16:29:04,888 Test: ../../../data-ceph/arguana/arg-generation/claim-target-tagger/data/ibm_ds/test_ds.tsv


In [6]:
label_type = 'ct'

label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

# 4. initialize fine-tuneable transformer embeddings WITH document context
embeddings = TransformerWordEmbeddings(model='xlm-roberta-large',
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=True,
                                       use_context=True,
                                       )

# 5. initialize bare-bones sequence tagger (no CRF, no RNN, no reprojection)
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type='ct',
                        use_crf=False,
                        use_rnn=False,
                        reproject_embeddings=False,
                        )

# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

# 7. run fine-tuning
trainer.fine_tune(model_folder,
                  learning_rate=5.0e-6,
                  mini_batch_size=4,
                  #mini_batch_chunk_size=1,  # remove this parameter to speed up computation if you have a big GPU
                  )

2022-05-01 16:29:07,968 Computing label dictionary. Progress:


1157it [00:00, 50027.93it/s]

2022-05-01 16:29:08,019 Dictionary created for label 'ct' with 2 values: CT (seen 1143 times)
Dictionary with 2 tags: <unk>, CT


2022-05-01 16:29:21,188 SequenceTagger predicts: Dictionary with 5 tags: O, S-CT, B-CT, E-CT, I-CT
2022-05-01 16:29:21,393 ----------------------------------------------------------------------------------------------------


/usr/local/lib/python3.6/dist-packages/flair/trainers/trainer.py:65: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  "There should be no best model saved at epoch 1 except there "


...

2022-05-01 16:36:07,213 Evaluating as a multi-label problem: False
2022-05-01 16:36:07,222 DEV : loss 0.5474958419799805 - f1-score (micro avg)  0.782
2022-05-01 16:36:07,226 BAD EPOCHS (no improvement): 4


2022-05-01 16:36:10,049 ----------------------------------------------------------------------------------------------------
2022-05-01 16:36:10,051 Testing using last state of model ...


100%|██████████| 244/244 [00:15<00:00, 15.57it/s]

2022-05-01 16:36:25,758 Evaluating as a multi-label problem: False
2022-05-01 16:36:25,769 0.7622	0.8079	0.7844	0.6452
2022-05-01 16:36:25,770 
Results:
- F-score (micro) 0.7844
- F-score (macro) 0.7844
- Accuracy 0.6452

By class:
              precision    recall  f1-score   support

          CT     0.7622    0.8079    0.7844       968

   micro avg     0.7622    0.8079    0.7844       968
   macro avg     0.7622    0.8079    0.7844       968
weighted avg     0.7622    0.8079    0.7844       968

2022-05-01 16:36:25,770 ----------------------------------------------------------------------------------------------------


{'test_score': 0.7843530591775327,
 'dev_score_history': [0.0,
  0.6783216783216782,
  0.7087719298245615,
  0.7311827956989249,
  0.7372262773722628,
  0.7636363636363634,
  0.7647058823529411,
  0.7794117647058822,
  0.7849056603773584,
  0.7819548872180451],
 'train_loss_history': [1.1407576502377446,
  0.6412852737494023,
  0.3791550337100714,
  0.2809379669566499,
  0.24066042675218985,
  0.22712329009040827,
  0.17782459732661007,
  0.17970192214440647,
  0.16248181861427086,
  0.1379217608272952],
 'dev_loss_history': [0.6870161890983582,
  0.49243220686912537,
  0.3301384150981903,
  0.4771294891834259,
  0.5390682816505432,
  0.5501019358634949,
  0.5791816711425781,
  0.511544406414032,
  0.5512567162513733,
  0.5474958419799805]}

### Extract targets from Reddit conclusions:

In [7]:
from flair.models import SequenceTagger
from flair.tokenization import SegtokSentenceSplitter

In [8]:
# predict tags for sentences
model = SequenceTagger.load(model_folder+'/final-model.pt')

2022-05-01 16:37:45,833 loading file ../../../data-ceph/arguana/arg-generation/claim-target-tagger/model/final-model.pt
2022-05-01 16:37:53,798 SequenceTagger predicts: Dictionary with 5 tags: O, S-CT, B-CT, E-CT, I-CT


In [9]:
def extract_targets(model, claims):
    sentences = [Sentence(x) for x in claims]
    model.predict(sentences)
    # iterate through sentences and print predicted labels
    targets = []
    for sentence in sentences:
        target_spans = sorted([(s.text, s.score) for s in sentence.get_spans('ct')], key=lambda x: -x[1])
        if len(target_spans) > 0:
            targets.append(target_spans[0][0])
        else:
            targets.append(sentence.to_original_text())
        
    return targets

In [10]:
def extract_targets_and_stances(df):
    unique_conclusions = df.title.unique().tolist()
    unique_conclusions_targets = extract_targets(model, unique_conclusions)
    unique_conclusions_stances = get_stances(unique_conclusions_targets, unique_conclusions)

    conc_to_targets = {x[0]: x[1] for x in zip(unique_conclusions, unique_conclusions_targets)}
    conc_to_stances = {x[0]: x[1] for x in zip(unique_conclusions, unique_conclusions_stances)}
    
    df['conclusion_targets'] = df.title.apply(lambda x: conc_to_targets[x])
    df['conclusion_stance']  = df.title.apply(lambda x: conc_to_stances[x])
    
    return df

----------------

In [23]:
#Extract conclusion target and stances for dev_sample_all
dev_df = pd.read_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/sample_valid_conclusion_all_preprocessed.pkl')

dev_df = dev_df[dev_df.title.str.len() > 0]
dev_df = extract_targets_and_stances(dev_df)
dev_df.to_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/sample_valid_conclusion_all_preprocessed.pkl')

ProConClient: 100%|██████████| 1997/1997 [00:33<00:00, 60.30it/s]


In [22]:
#Extract conclusion target and stances for test_sample_all
test_df = pd.read_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/sample_test_conclusion_all_preprocessed.pkl')

test_df = test_df[test_df.title.str.len() > 0]
test_df = extract_targets_and_stances(test_df)
test_df.to_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/sample_test_conclusion_all_preprocessed.pkl')

ProConClient: 100%|██████████| 2000/2000 [00:35<00:00, 56.97it/s]
